In [3]:
import sys
sys.path.append('../')
from dataproc import extract_wvs
from dataproc import get_discharge_summaries
from dataproc import concat_and_split
from dataproc import build_vocab
from dataproc import vocab_index_descriptions
from dataproc import word_embeddings
from constants import MIMIC_4_DIR, MIMIC_4_NOTES_DIR, MIMIC_4_SAVE_DIR

import numpy as np
import pandas as pd

from collections import Counter
import csv
import operator

Let's do some data processing in a much better way, with a notebook.

First, let's define some stuff.

In [2]:
Y = 'full' #use all available labels in the dataset for prediction
notes_file = f'{MIMIC_4_NOTES_DIR}/note/discharge.csv' # raw note events downloaded from MIMIC-III
vocab_size = 'full' #don't limit the vocab size to a specific number
vocab_min = 3 #discard tokens appearing in fewer than this many documents

# Data processing

## Combine diagnosis and procedure codes and reformat them

The codes in MIMIC-III are given in separate files for procedures and diagnoses, and the codes are given without periods, which might lead to collisions if we naively combine them. So we have to add the periods back in the right place.

In [ ]:
dfproc = pd.read_csv(f'{MIMIC_4_DIR}/hosp/procedures_icd.csv.gz', compression="gzip",
                     dtype={"icd_code": str})
dfdiag = pd.read_csv(f'{MIMIC_4_DIR}/hosp/diagnoses_icd.csv.gz', compression="gzip",
                     dtype={"icd_code": str})

In [ ]:
dfproc=dfproc[dfproc['icd_version']==10]
dfdiag=dfdiag[dfdiag['icd_version']==10]

In [ ]:
dfdiag['absolute_code'] = dfdiag['icd_code']
dfproc['absolute_code'] = dfproc['icd_code']

In [ ]:
dfcodes = pd.concat([dfdiag, dfproc])

In [ ]:
dfcodes.rename(columns={"subject_id": "SUBJECT_ID",
                        "hadm_id": "HADM_ID",
                        "seq_num": "SEQ_NUM",
                        "icd_code": "ICD_CODE"},
                        inplace=True
                        )

In [ ]:
dfcodes.to_csv('%s/ALL_CODES.csv' % MIMIC_4_SAVE_DIR, index=False,
               columns=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'absolute_code'],
               header=['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD_CODE'])

## How many codes are there?

In [ ]:
#In the full dataset (not just discharge summaries)
df = pd.read_csv('%s/ALL_CODES.csv' % MIMIC_4_SAVE_DIR, dtype={"ICD_CODE": str})
len(df['ICD_CODE'].unique())

## Tokenize and preprocess raw text

Preprocessing time!

This will:
- Select only discharge summaries and their addenda
- remove punctuation and numeric-only tokens, removing 500 but keeping 250mg
- lowercase all tokens

In [34]:
import importlib

In [ ]:
# Import the outer module
outer_module = importlib.import_module('dataproc')

# Get the inner module from the outer module
inner_module = getattr(outer_module, 'get_discharge_summaries')

# Reload the inner module
get_discharge_summaries = importlib.reload(inner_module)

In [ ]:
#This reads all notes, selects only the discharge summaries, and tokenizes them, returning the output filename
disch_full_file = get_discharge_summaries.write_discharge_summaries(out_file="%s/disch_full.csv" % MIMIC_4_SAVE_DIR)

Let's read this in and see what kind of data we're working with

In [14]:
df = pd.read_csv('%s/disch_full.csv' % MIMIC_4_SAVE_DIR)

In [15]:
#How many admissions?
len(df['HADM_ID'].unique())

331794

In [16]:
df

,SUBJECT_ID,HADM_ID,CHARTTIME,TEXT
0,10000032,22595853,2180-05-07 00:00:00,name ___ unit no ___ admission date ___ discha...
1,10000032,22841357,2180-06-27 00:00:00,name ___ unit no ___ admission date ___ discha...
2,10000032,25742920,2180-08-07 00:00:00,name ___ unit no ___ admission date ___ discha...
3,10000032,29079034,2180-07-25 00:00:00,name ___ unit no ___ admission date ___ discha...
4,10000084,23052089,2160-11-25 00:00:00,name ___ unit no ___ admission date ___ discha...
...,...,...,...,...
331789,19999828,25744818,2149-01-18 00:00:00,name ___ unit no ___ admission date ___ discha...
331790,19999828,29734428,2147-08-04 00:00:00,name ___ unit no ___ admission date ___ discha...
331791,19999840,21033226,2164-09-17 00:00:00,name ___ unit no ___ admission date ___ discha...
331792,19999840,26071774,2164-07-28 00:00:00,name ___ unit no ___ admission date ___ discha...


In [ ]:
#Tokens and types
types = set()
num_tok = 0
for row in df.itertuples():
    for w in row[4].split():
        types.add(w)
        num_tok += 1

In [ ]:
print("Num types", len(types))
print("Num tokens", str(num_tok))

In [17]:
#Let's sort by SUBJECT_ID and HADM_ID to make a correspondence with the MIMIC-3 label file
df = df.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [18]:
#Sort the label file by the same
dfl = pd.read_csv('%s/ALL_CODES.csv' % MIMIC_4_SAVE_DIR)
dfl = dfl.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [9]:
len(df['HADM_ID'].unique()), len(dfl['HADM_ID'].unique())

(331794, 154076)

## Consolidate labels with set of discharge summaries

Looks like there were some HADM_ID's that didn't have discharge summaries, so they weren't included with our notes

In [20]:
#Let's filter out these HADM_ID's
hadm_ids = set(df['HADM_ID'])
with open('%s/ALL_CODES.csv' % MIMIC_4_SAVE_DIR, 'r') as lf:
    with open('%s/ALL_CODES_filtered.csv' % MIMIC_4_SAVE_DIR, 'w') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'ICD_CODE', 'ADMITTIME', 'DISCHTIME'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[1])
            #print(hadm_id)
            #break
            if hadm_id in hadm_ids:
                w.writerow(row[:2] + [row[-1], '', ''])

In [21]:
dfl = pd.read_csv('%s/ALL_CODES_filtered.csv' % MIMIC_4_SAVE_DIR, index_col=None)

In [22]:
len(dfl['HADM_ID'].unique())

122317

In [25]:
#Let's filter out these HADM_ID in the note but not in the label
hadm_lids = set(dfl['HADM_ID'])
with open(f'{MIMIC_4_SAVE_DIR}/disch_full.csv', 'r', encoding='utf-8') as lf:
    with open(f'{MIMIC_4_SAVE_DIR}/disch_full_filtered.csv','w', encoding='utf-8') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'TEXT'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[1])
            #print(hadm_id)
            #break
            if hadm_id in hadm_lids:
                w.writerow(row)

In [27]:
#we still need to sort it by HADM_ID
dfl = dfl.sort_values(['SUBJECT_ID', 'HADM_ID'])
dfl.to_csv('%s/ALL_CODES_filtered.csv' % MIMIC_4_SAVE_DIR, index=False)

## Append labels to notes in a single file

In [29]:
#Now let's append each instance with all of its codes
#this is pretty non-trivial so let's use this script I wrote, which requires the notes to be written to file
df = pd.read_csv(f'{MIMIC_4_SAVE_DIR}/disch_full_filtered.csv', index_col=None, encoding='utf-8', engine='python')
df = df.sort_values(['SUBJECT_ID', 'HADM_ID'])
sorted_file = f'{MIMIC_4_SAVE_DIR}/disch_full_filtered.csv'
df.to_csv(sorted_file, index=False, encoding='utf-8')

In [35]:
# Import the outer module
outer_module = importlib.import_module('dataproc')

# Get the inner module from the outer module
inner_module = getattr(outer_module, 'concat_and_split')

# Reload the inner module
concat_and_split = importlib.reload(inner_module)

In [36]:
labeled = concat_and_split.concat_data('%s/ALL_CODES_filtered.csv' % MIMIC_4_SAVE_DIR, sorted_file)

CONCATENATING
0 done
10000 done
20000 done
30000 done
40000 done
50000 done
60000 done
70000 done
80000 done
90000 done
100000 done
110000 done
120000 done


In [37]:
df.sort_values(['SUBJECT_ID', 'HADM_ID'])

,SUBJECT_ID,HADM_ID,CHARTTIME,TEXT
0,10000084,23052089,2160-11-25 00:00:00,name ___ unit no ___ admission date ___ discha...
1,10000117,22927623,2181-11-15 00:00:00,name ___ unit no ___ admission date ___ discha...
2,10000117,27988844,2183-09-21 00:00:00,name ___ unit no ___ admission date ___ discha...
3,10000980,20897796,2193-08-17 00:00:00,name ___ unit no ___ admission date ___ discha...
4,10000980,25911675,2191-05-24 00:00:00,name ___ unit no ___ admission date ___ discha...
...,...,...,...,...
122312,19999784,29355057,2119-10-23 00:00:00,name ___ unit no ___ admission date ___ discha...
122313,19999784,29889147,2120-10-31 00:00:00,name ___ unit no ___ admission date ___ discha...
122314,19999784,29956342,2121-02-05 00:00:00,name ___ unit no ___ admission date ___ discha...
122315,19999828,25744818,2149-01-18 00:00:00,name ___ unit no ___ admission date ___ discha...


In [38]:
dfl.sort_values(['SUBJECT_ID', 'HADM_ID'])

,SUBJECT_ID,HADM_ID,ICD_CODE,ADMITTIME,DISCHTIME
0,10000084,23052089,G3183,NaN,NaN
1,10000084,23052089,F0280,NaN,NaN
2,10000084,23052089,R441,NaN,NaN
3,10000084,23052089,R296,NaN,NaN
4,10000084,23052089,E785,NaN,NaN
...,...,...,...,...,...
1974461,19999828,29734428,0HR7X74,NaN,NaN
1974462,19999828,29734428,0HBJXZZ,NaN,NaN
1974463,19999828,29734428,0HBHXZZ,NaN,NaN
1974464,19999828,29734428,02HV33Z,NaN,NaN


In [39]:
#name of the file we just made
print(labeled)

/media/ssd-3t-2/isviridov/mimic4/physionet.org/files/mimiciv/2.2/notes_labeled.csv


Let's sanity check the combined data we just made. Do we have all hadm id's accounted for, and the same vocab stats?

In [40]:
dfnl = pd.read_csv(labeled)
#Tokens and types
types = set()
num_tok = 0
for row in dfnl.itertuples():
    for w in row[3].split():
        types.add(w)
        num_tok += 1

In [41]:
print("num types", len(types), "num tokens", num_tok)

num types 188781 num tokens 203251866


In [42]:
len(dfnl['HADM_ID'].unique())

122317

## Create train/dev/test splits

In [43]:
# Import the outer module
outer_module = importlib.import_module('dataproc')

# Get the inner module from the outer module
inner_module = getattr(outer_module, 'concat_and_split')

# Reload the inner module
concat_and_split = importlib.reload(inner_module)

In [44]:
fname = '%s/notes_labeled.csv' % MIMIC_4_SAVE_DIR
base_name = "%s/disch" % MIMIC_4_SAVE_DIR #for output
tr, dv, te = concat_and_split.split_data(fname, base_name=base_name)

SPLITTING
0 read
10000 read
20000 read
30000 read
40000 read
50000 read
60000 read
70000 read
80000 read
90000 read
100000 read
110000 read
120000 read


## Build vocabulary from training data

In [45]:
vocab_min = 3
vname = '%s/vocab.csv' % MIMIC_4_SAVE_DIR
build_vocab.build_vocab(vocab_min, tr, vname)

reading in data...
removing rare terms
69972 terms qualify out of 179131 total
writing output


## Sort each data split by length for batching

In [46]:
for splt in ['train', 'dev', 'test']:
    filename = '%s/disch_%s_split.csv' % (MIMIC_4_SAVE_DIR, splt)
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_full.csv' % (MIMIC_4_SAVE_DIR, splt), index=False)

## Pre-train word embeddings

Let's train word embeddings on all words

In [59]:
# Import the outer module
outer_module = importlib.import_module('dataproc')

# Get the inner module from the outer module
inner_module = getattr(outer_module, 'vocab_index_descriptions')

# Reload the inner module
vocab_index_descriptions = importlib.reload(inner_module)

In [51]:
w2v_file = word_embeddings.word_embeddings('full', '%s/disch_full.csv' % MIMIC_4_SAVE_DIR, 100, 0, 5)

building word2vec vocab on /media/ssd-3t-2/isviridov/mimic4/orig_mimic_prepared/disch_full.csv...
training...
writing embeddings to /media/ssd-3t-2/isviridov/mimic4/orig_mimic_prepared/processed_full.w2v


## Write pre-trained word embeddings with new vocab

In [56]:
extract_wvs.gensim_to_embeddings('%s/processed_full.w2v' % MIMIC_4_SAVE_DIR, '%s/vocab.csv' % MIMIC_4_SAVE_DIR, Y)

100%|██████████| 69972/69972 [00:00<00:00, 130011.40it/s]


## Pre-process code descriptions using the vocab

In [60]:
vocab_index_descriptions.vocab_index_descriptions('%s/vocab.csv' % MIMIC_4_SAVE_DIR,
                                                  '%s/description_vectors.vocab' % MIMIC_4_SAVE_DIR)

AttributeError: module 'datasets' has no attribute 'load_code_descriptions'

## Filter each split to the top 50 diagnosis/procedure codes

In [61]:
Y = 50

In [62]:
#first calculate the top k
counts = Counter()
dfnl = pd.read_csv('%s/notes_labeled.csv' % MIMIC_4_SAVE_DIR)
for row in dfnl.itertuples():
    for label in str(row[4]).split(';'):
        counts[label] += 1

In [63]:
codes_50 = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)

In [64]:
codes_50 = [code[0] for code in codes_50[:Y]]

In [65]:
codes_50

['E785',
 'I10',
 'Z87891',
 'K219',
 'F329',
 'I2510',
 'N179',
 'F419',
 'Z7901',
 'Z794',
 'E039',
 'E119',
 'G4733',
 'D649',
 'E669',
 'I4891',
 'F17210',
 'Y929',
 'Z66',
 'J45909',
 'Z7902',
 'J449',
 'D62',
 '02HV33Z',
 'N390',
 'I129',
 'E1122',
 'E871',
 'I252',
 'N189',
 'E872',
 'Z8673',
 'Z955',
 'Z86718',
 'G8929',
 'I110',
 'K5900',
 'N400',
 'N183',
 'I480',
 'I130',
 'G4700',
 'D696',
 'Z951',
 'M109',
 'Y92239',
 'J9601',
 'J189',
 'Z23',
 'Y92230']

In [66]:
with open('%s/TOP_%s_CODES.csv' % (MIMIC_4_SAVE_DIR, str(Y)), 'w') as of:
    w = csv.writer(of)
    for code in codes_50:
        w.writerow([code])

In [68]:
for splt in ['train', 'dev', 'test']:
    print(splt)
    hadm_ids = set()
    with open('%s/%s_50_hadm_ids.csv' % (MIMIC_4_SAVE_DIR, splt), 'r') as f:
        for line in f:
            hadm_ids.add(line.rstrip())
    with open('%s/notes_labeled.csv' % MIMIC_4_SAVE_DIR, 'r') as f:
        with open('%s/%s_%s.csv' % (MIMIC_4_SAVE_DIR, splt, str(Y)), 'w') as of:
            r = csv.reader(f)
            w = csv.writer(of)
            #header
            w.writerow(next(r))
            i = 0
            for row in r:
                hadm_id = row[1]
                if hadm_id not in hadm_ids:
                    continue
                codes = set(str(row[3]).split(';'))
                filtered_codes = codes.intersection(set(codes_50))
                if len(filtered_codes) > 0:
                    w.writerow(row[:3] + [';'.join(filtered_codes)])
                    i += 1

train


dev
test


In [69]:
for splt in ['train', 'dev', 'test']:
    filename = '%s/%s_%s.csv' % (MIMIC_4_SAVE_DIR, splt, str(Y))
    df = pd.read_csv(filename)
    df['length'] = df.apply(lambda row: len(str(row['TEXT']).split()), axis=1)
    df = df.sort_values(['length'])
    df.to_csv('%s/%s_%s.csv' % (MIMIC_4_SAVE_DIR, splt, str(Y)), index=False)

In [70]:
### ПЕРЕВЕСТИ DATAFRAME в JSON БЕЗ ADDITION ###